In [20]:
import os
import glob
import nltk
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import torch
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, Trainer, TrainingArguments
import numpy as np
from sklearn.metrics import classification_report, accuracy_score, precision_recall_fscore_support
import requests
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings("ignore")
import re 
from sklearn.pipeline import Pipeline

In [3]:
# Реализация 1 задания
# Загрузка данных

data_path = 'bbc'  

categories = ['business', 'entertainment', 'politics', 'sport', 'tech']

data = []
for category in categories:
    folder_path = os.path.join(data_path, category)
    files = glob.glob(os.path.join(folder_path, '*.txt'))
    for filepath in files:
        with open(filepath, 'r', encoding='utf-8', errors='ignore') as f:
            text = f.read()
        data.append({'text': text, 'label': category})

df = pd.DataFrame(data)

# Разделение на обучающую и тестовую выборки

X_train, X_test, y_train, y_test = train_test_split(df['text'], df['label'], test_size=0.2, random_state=42)
print(f"Размер обучающей выборки: {len(X_train)}")
print(f"Размер тестовой выборки: {len(X_test)}")


Размер обучающей выборки: 1776
Размер тестовой выборки: 445


In [4]:
# Векторизация текста (TF-IDF)

tfidf_vectorizer = TfidfVectorizer(stop_words='english') # Удаляем стоп-слова
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train) # Обучаем и преобразуем обучающую выборку
X_test_tfidf = tfidf_vectorizer.transform(X_test) # Преобразуем тестовую выборку

In [5]:
# Обучение Multinomial Naive Bayes модели 

model = MultinomialNB()
model.fit(X_train_tfidf, y_train)

MultinomialNB()

In [6]:
# Оценка модели

y_pred = model.predict(X_test_tfidf)
print(classification_report(y_test, y_pred))

               precision    recall  f1-score   support

     business       0.97      0.96      0.97       110
entertainment       0.98      0.93      0.96        70
     politics       0.93      0.96      0.95        82
        sport       0.98      1.00      0.99        94
         tech       0.97      0.97      0.97        89

     accuracy                           0.97       445
    macro avg       0.97      0.96      0.97       445
 weighted avg       0.97      0.97      0.97       445



In [7]:
#  Пример инференса

def predict_category(text):
    text_tfidf = tfidf_vectorizer.transform([text])  # Преобразуем текст в TF-IDF вектор
    predicted_category = model.predict(text_tfidf)[0] # Получаем предсказанную категорию
    return predicted_category

# Пример использования:
new_text = "This article is about volleyball team victory"
predicted_category = predict_category(new_text)
print(f"Предсказанная категория для нового текста: {predicted_category}")

new_text = "This article is about a company's profits and financial performance."
predicted_category = predict_category(new_text)
print(f"Предсказанная категория для нового текста: {predicted_category}")

new_text = "The new film won Oscars awards!"
predicted_category = predict_category(new_text)
print(f"Предсказанная категория для нового текста: {predicted_category}")

Предсказанная категория для нового текста: sport
Предсказанная категория для нового текста: business
Предсказанная категория для нового текста: entertainment


In [8]:
# Реализация 2 задания
#  Подготовка данных для DistilBERT
# Кодируем текстовые метки классов в числовые

label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)


# Загружаем токенизатор DistilBERT

tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

# Токенизируем данные

train_encodings = tokenizer(X_train.tolist(), truncation=True, padding=True)
test_encodings = tokenizer(X_test.tolist(), truncation=True, padding=True)

# Определяем класс Dataset для Transformers

class NewsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = NewsDataset(train_encodings, y_train_encoded)
test_dataset = NewsDataset(test_encodings, y_test_encoded)


In [9]:
# Обучение модели DistilBERT

# Загружаем модель DistilBERT для классификации последовательностей

model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=len(categories))

# Определяем метрику для оценки

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }
# Определяем аргументы обучения

training_args = TrainingArguments(
    output_dir='./results',          # Каталог для сохранения модели
    evaluation_strategy="epoch",     # Оценка после каждой эпохи
    save_strategy="epoch",          # Сохранение после каждой эпохи
    num_train_epochs=3,              # Количество эпох обучения (можете увеличить)
    per_device_train_batch_size=16,   # Размер батча для обучения (можно настроить)
    per_device_eval_batch_size=64,    # Размер батча для оценки
    warmup_steps=500,                # Количество шагов для "разогрева" learning rate
    weight_decay=0.01,               # Weight decay
    logging_dir='./logs',            # Каталог для логирования
    logging_steps=10,
    load_best_model_at_end=True,       # Загружать лучшую модель в конце обучения
    metric_for_best_model='accuracy',  # Метрика для определения лучшей модели
)

#  Определяем Trainer

trainer = Trainer(
    model=model, 
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)
# Обучаем модель

trainer.train()

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.733900,0.639509,0.961798,0.961807,0.962836,0.961798
2,0.040600,0.110365,0.973034,0.972942,0.974131,0.973034
3,0.063400,0.076580,0.977528,0.977526,0.977573,0.977528


TrainOutput(global_step=333, training_loss=0.5440685008135107, metrics={'train_runtime': 17739.1259, 'train_samples_per_second': 0.3, 'train_steps_per_second': 0.019, 'total_flos': 705824060129280.0, 'train_loss': 0.5440685008135107, 'epoch': 3.0})

In [10]:
# Оценка модели DistilBERT

predictions = trainer.predict(test_dataset)
y_pred_bert = np.argmax(predictions.predictions, axis=-1)
print("DistilBERT Classification Report:\n", classification_report(y_test_encoded, y_pred_bert, target_names=categories))


DistilBERT Classification Report:
                precision    recall  f1-score   support

     business       0.97      0.96      0.97       110
entertainment       0.99      1.00      0.99        70
     politics       0.96      0.96      0.96        82
        sport       1.00      0.99      0.99        94
         tech       0.97      0.98      0.97        89

     accuracy                           0.98       445
    macro avg       0.98      0.98      0.98       445
 weighted avg       0.98      0.98      0.98       445



In [22]:
# реализуем третье задание
# Создаем пайплайн
pipeline_nb = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words='english')),
    ('classifier', MultinomialNB())
])

# Обучаем модель
pipeline_nb.fit(X_train, y_train)

# Проверяем качество на тестовой выборке
y_pred_nb = pipeline_nb.predict(X_test)
print("Multinomial Naive Bayes Classification Report:\n", classification_report(y_test, y_pred_nb))


Multinomial Naive Bayes Classification Report:
                precision    recall  f1-score   support

     business       0.97      0.96      0.97       110
entertainment       0.98      0.93      0.96        70
     politics       0.93      0.96      0.95        82
        sport       0.98      1.00      0.99        94
         tech       0.97      0.97      0.97        89

     accuracy                           0.97       445
    macro avg       0.97      0.96      0.97       445
 weighted avg       0.97      0.97      0.97       445



In [23]:
# Пайплайн инференса Multinomial Naive Bayes, возвращает метку класса и вероятности принадлежности к каждому классу
def nb_inference_pipeline(text):
    label = pipeline_nb.predict([text])[0]
    probabilities = pipeline_nb.predict_proba([text])[0]
    return label, probabilities

In [24]:
# Пайплайн Инференса DistilBERT, , возвращает метку класса и вероятности принадлежности к каждому классу
def bert_inference_pipeline(text):
    encoding = tokenizer(text, truncation=True, padding=True, return_tensors='pt')
    with torch.no_grad():
      outputs = model(**encoding)

    logits = outputs.logits
    probabilities = torch.nn.functional.softmax(logits, dim=-1).cpu().numpy()[0]
    predicted_class_id = np.argmax(probabilities)
    predicted_label = label_encoder.inverse_transform([predicted_class_id])[0]

    return predicted_label, probabilities

In [28]:
# Сбор свежих новостей с BBC News
def scrape_bbc_news(category, num_articles=5):
    """
    Собирает указанное количество свежих новостей из указанной категории BBC News.
    Адаптировано для новой структуры сайта.
    Возвращает список словарей с текстом новости и URL.
    """
    category_urls = {
        'business': 'https://www.bbc.com/news/business',
        'entertainment': 'https://www.bbc.com/news/entertainment_and_arts',
        'politics': 'https://www.bbc.com/news/politics',
        'sport': 'https://www.bbc.com/sport',
        'tech': 'https://www.bbc.com/news/technology'
    }

    if category not in category_urls:
        print(f"Ошибка: Категория '{category}' не найдена в списке URL.")
        return []

    category_url = category_urls[category]

    try:
        response = requests.get(category_url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')

        articles = []
        count = 0

        # Находим все ссылки на статьи
        links = []
        if category != 'sport':
            links = soup.find_all('a', class_='sc-2e6baa30-0 gILusN')  # Используем класс ссылки для других категорий
        else:
            # Если это категория "sport", используем новый селектор
            links = soup.find_all('a', class_='ssrcss-1oo1bfh-PromoLink exn3ah95')


        for link in links:
            href = link['href']

            # Проверяем, является ли ссылка относительной или абсолютной
            if href.startswith('/news/'):  # Проверяем, начинается ли с '/news/'
                article_url = "https://www.bbc.com" + href
            elif href.startswith('/sport/'):  # Проверяем, начинается ли с '/sport/' (для спорта)
                article_url = "https://www.bbc.com" + href
            elif href.startswith('https://www.bbc.com/'):  # Проверяем, начинается ли с 'https://www.bbc.com/'
                article_url = href
            else:
                # Если это не ссылка на статью, пропускаем ее
                print(f"Пропущена ссылка (не статья): {href}")
                continue

            # Дополнительная проверка, чтобы убедиться, что это ссылка на статью
            if not re.search(r'/(news|sport)/\w+', article_url):  # Используем регулярное выражение
                print(f"Пропущена ссылка (не статья, не соответствует шаблону): {article_url}")
                continue

            try:
                article_response = requests.get(article_url)
                article_response.raise_for_status()
                article_soup = BeautifulSoup(article_response.content, 'html.parser')
                article_text = ''

                # Попытка 1: Извлечение текста из div с data-component="text-block"
                text_block = article_soup.find('div', {'data-component': 'text-block'})
                if text_block:
                    for p in text_block.find_all('p'):
                        article_text += p.text + '\n'

                # Попытка 2: Если не найдено, извлечение текста из div с class="article__body"
                if not article_text:
                    article_body = article_soup.find('div', class_='article__body')
                    if article_body:
                        for p in article_body.find_all('p'):
                            article_text += p.text + '\n'

                # Попытка 3: Если не найдено и это страница спорта, ищем div с class="ssrcss-11r1m41-RichTextComponentWrapper e24u68q0"
                if category == 'sport' and not article_text:
                    sport_text_wrapper = article_soup.find('div', class_='ssrcss-11r1m41-RichTextComponentWrapper e24u68q0')
                    if sport_text_wrapper:
                        for p in sport_text_wrapper.find_all('p'):
                            article_text += p.text + '\n'

                if article_text:
                    articles.append({'text': article_text, 'url': article_url})
                    count += 1
                else:
                    print(f"Предупреждение: Пустой текст статьи с URL: {article_url}")

            except requests.exceptions.RequestException as e:
                print(f"Ошибка при запросе статьи {article_url}: {e}")

            if count >= num_articles:
                break

        return articles

    except requests.exceptions.RequestException as e:
        print(f"Ошибка при запросе категории {category_url}: {e}")
        return []

# Собираем свежие новости для каждой категории
categories = ['business', 'entertainment', 'politics', 'sport', 'tech']
fresh_news = {}
for category in categories:
    fresh_news[category] = scrape_bbc_news(category, num_articles=1)  # Сократим до 1 для быстроты

# Выводим собранные новости (включая полные URL и текст)
for category, articles in fresh_news.items():
    print(f"Категория: {category}")
    if articles:
        for article in articles:
            print(f"URL: {article['url']}")
            print(f"Текст: {article['text'][:500]}...")  # Выводим первые 500 символов текста
    else:
        print(f"  Не удалось получить новости для категории {category}")
    print("-" * 50)

Пропущена ссылка (не статья): /
Пропущена ссылка (не статья): /
Пропущена ссылка (не статья): /sport
Пропущена ссылка (не статья): /video
Пропущена ссылка (не статья, не соответствует шаблону): https://www.bbc.com/weather
Пропущена ссылка (не статья, не соответствует шаблону): https://www.bbc.com/newsletters
Пропущена ссылка (не статья): /
Пропущена ссылка (не статья): /
Пропущена ссылка (не статья): /sport
Пропущена ссылка (не статья): /video
Пропущена ссылка (не статья, не соответствует шаблону): https://www.bbc.com/weather
Пропущена ссылка (не статья, не соответствует шаблону): https://www.bbc.com/newsletters
Пропущена ссылка (не статья): /
Пропущена ссылка (не статья): /
Пропущена ссылка (не статья): /sport
Пропущена ссылка (не статья): /video
Пропущена ссылка (не статья, не соответствует шаблону): https://www.bbc.com/weather
Пропущена ссылка (не статья, не соответствует шаблону): https://www.bbc.com/newsletters
Пропущена ссылка (не статья): /
Пропущена ссылка (не статья): /
Пропущ

In [29]:
# Инференс и оценка пайплайнов на свежих данных ---

# Запускаем инференс пайплайнов на свежих данных и оцениваем результаты
results = {}
for category, articles in fresh_news.items():
    results[category] = {
        'nb': [],
        'bert': []
    }
    for article in articles:
        text = article['text']
        url = article['url']

        # Inference with Naive Bayes pipeline
        label_nb, probabilities_nb = nb_inference_pipeline(text)
        results[category]['nb'].append({
            'url': url,
            'predicted_label': label_nb,
            'probabilities': probabilities_nb
        })

        # Inference with DistilBERT pipeline
        label_bert, probabilities_bert = bert_inference_pipeline(text)
        results[category]['bert'].append({
            'url': url,
            'predicted_label': label_bert,
            'probabilities': probabilities_bert
        })

In [30]:
# Выводим результаты инференс
for category, pipeline_results in results.items():
    print(f"Категория: {category}")
    print("-" * 50)
    for pipeline, articles in pipeline_results.items():
        print(f"Пайплайн: {pipeline}")
        for article in articles:
            print(f"  URL: {article['url']}")
            print(f"  Predicted Label: {article['predicted_label']}")
            print(f"  Probabilities: {article['probabilities']}")
        print("-" * 30)

# Общая оценка (пример) ---
correct_nb = 0
correct_bert = 0
total = 0

for category, pipeline_results in results.items():
    for pipeline, articles in pipeline_results.items():
        for article in articles:
            total += 1
            if pipeline == 'nb' and article['predicted_label'] == category:
                correct_nb += 1
            if pipeline == 'bert' and article['predicted_label'] == category:
                correct_bert += 1

print(f"Accuracy Naive Bayes: {correct_nb/total}")
print(f"Accuracy Bert: {correct_bert/total}")

Категория: business
--------------------------------------------------
Пайплайн: nb
  URL: https://www.bbc.com/news/articles/cgrgj51gl25o
  Predicted Label: business
  Probabilities: [0.71648565 0.0522092  0.11173555 0.05315651 0.0664131 ]
------------------------------
Пайплайн: bert
  URL: https://www.bbc.com/news/articles/cgrgj51gl25o
  Predicted Label: business
  Probabilities: [0.99381554 0.00135285 0.00176172 0.00127502 0.00179493]
------------------------------
Категория: entertainment
--------------------------------------------------
Пайплайн: nb
  URL: https://www.bbc.com/news/articles/cn7v7v2r3n2o
  Predicted Label: entertainment
  Probabilities: [0.08118021 0.47698221 0.1040774  0.21067577 0.12708441]
------------------------------
Пайплайн: bert
  URL: https://www.bbc.com/news/articles/cn7v7v2r3n2o
  Predicted Label: entertainment
  Probabilities: [0.00186294 0.99281317 0.00181667 0.00159135 0.00191573]
------------------------------
Категория: politics
-------------------

In [ ]:
'''очень высокая вероятность bert для бизнеса - 0,99 (nb - 0,71)
   очень высокая вероятность bert для entertainment - 0,99 (nb - 0,48)
   очень высокая вероятность bert для sport - 0,99 (nb тоже высокая - 0,91)
   очень высокая вероятность bert для tech - 0,99 (nb тоже  - 0,69)
   однако и bert и nb ошибочно предсказали класс politics как business и при этом bert ошиблась сильнее, 
   что, скорее всего связано с близостью тем бизнеса и политики
'''